In [86]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, RidgeCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from math import sqrt

# LGB
import lightgbm as lgb
#Read multiple files
import glob

In [87]:
# Define categorical columns
categorical_columns = ['Vehicle_Performance','pol_zip_code',
                       'Vehicle_Passive_Restraint',
                       'Policy_Company', # 'Policy_Billing_Code',
                       'Policy_Method_Of_Payment',
                       'Policy_Reinstatement_Fee_Indicator',
                       'Vehicle_Comprehensive_Coverage_Indicator',
                       'Vehicle_Collision_Coverage_Indicator',
#                        'Vehicle_Youthful_Driver_Indicator',
                        'Vehicle_Youthful_Good_Student_Code',
#                       'Vehicle_Youthful_Driver_Training_Code',
                     'Vehicle_Safe_Driver_Discount_Indicator',
#                     'EEA_Liability_Coverage_Only_Indicator',
                     'EEA_Multi_Auto_Policies_Indicator',
                     'EEA_Packaged_Policy_Indicator',
                     'EEA_Full_Coverage_Indicator',
                       'EEA_Agency_Type',
                     'SYS_Renewed',
                     'SYS_New_Business',
                      'Vehicle_Usage',
#                      'Vehicle_Anti_Theft_Device',
                       'Vehicle_Make_Description']

In [88]:
df_schm = pd.read_csv('Balance_training.csv')
df_schm.shape

df_schm['Vehicle_Performance']= df_schm['Vehicle_Performance'].str.replace(" ","")
df_schm['Policy_Billing_Code']= df_schm['Policy_Billing_Code'].str.replace(" ","")
df_schm['Vehicle_Youthful_Good_Student_Code']= df_schm['Vehicle_Youthful_Good_Student_Code'].str.replace(" ","")
df_schm['Vehicle_Anti_Theft_Device']= df_schm['Vehicle_Anti_Theft_Device'].str.replace(" ","")
df_schm['Vehicle_Anti_Theft_Device']= df_schm['Vehicle_Anti_Theft_Device'].str.replace("-","")
df_schm['Vehicle_Youthful_Driver_Training_Code']= df_schm['Vehicle_Youthful_Driver_Training_Code'].str.replace(" ","")
df_schm['Policy_Method_Of_Payment']= df_schm['Policy_Method_Of_Payment'].str.replace("-","")
df_schm['EEA_Agency_Type']= df_schm['EEA_Agency_Type'].str.replace("-","")



In [89]:
df_schm[['Vehicle_Make_Year','Vehicle_Bodily_Injury_Limit','Driver_Total_Licensed_In_State','EEA_Prior_Bodily_Injury_Limit','Policy_Installment_Term','Vehicle_Youthful_Driver_Indicator','EEA_Liability_Coverage_Only_Indicator','Policy_Billing_Code','Vehicle_Youthful_Driver_Training_Code','Vehicle_Anti_Theft_Device']].head()

,Vehicle_Make_Year,Vehicle_Bodily_Injury_Limit,Driver_Total_Licensed_In_State,EEA_Prior_Bodily_Injury_Limit,Policy_Installment_Term,Vehicle_Youthful_Driver_Indicator,EEA_Liability_Coverage_Only_Indicator,Policy_Billing_Code,Vehicle_Youthful_Driver_Training_Code,Vehicle_Anti_Theft_Device
0,2006,25000,1,25000,6,N,N,DirectBilledtoInsured,NotApplicable,NotApplicable
1,2003,50000,1,50000,6,N,N,DirectBilledtoInsured,NotApplicable,AlarmOnly
2,1993,25000,1,25000,6,N,Y,DirectBilledtoInsured,NotApplicable,NotApplicable
3,2005,25000,1,25000,6,N,N,DirectBilledtoInsured,NotApplicable,PassiveDisablingVehicleRecovery
4,1993,35000,1,35000,6,N,Y,DirectBilledtoInsured,NotApplicable,NotApplicable


In [90]:
df_schm.drop(['Vehicle_Make_Year','EEA_Policy_Zip_Code_3','Vehicle_Bodily_Injury_Limit','Driver_Total_Licensed_In_State','EEA_Prior_Bodily_Injury_Limit','Policy_Installment_Term','Vehicle_Youthful_Driver_Indicator','EEA_Liability_Coverage_Only_Indicator','Policy_Billing_Code','Vehicle_Youthful_Driver_Training_Code','Vehicle_Anti_Theft_Device'],axis=1, inplace=True)

In [91]:
df_schm = pd.get_dummies(df_schm, columns=categorical_columns, prefix_sep="_")

In [92]:
path = r'.\PortfoliowisePredict' # use your path
all_files = glob.glob(path + "/*.csv")
X_mean = pd.DataFrame()
Y_mean = pd.DataFrame()

for filename in all_files:
    df_train = pd.read_csv(filename)
    
    df_train['Vehicle_Performance']= df_train['Vehicle_Performance'].str.replace(" ","")
    df_train['Policy_Billing_Code']= df_train['Policy_Billing_Code'].str.replace(" ","")
    df_train['Vehicle_Youthful_Good_Student_Code']= df_train['Vehicle_Youthful_Good_Student_Code'].str.replace(" ","")
    df_train['Vehicle_Anti_Theft_Device']= df_train['Vehicle_Anti_Theft_Device'].str.replace(" ","")
    df_train['Vehicle_Anti_Theft_Device']= df_train['Vehicle_Anti_Theft_Device'].str.replace("-","")
    df_train['Vehicle_Youthful_Driver_Training_Code']= df_train['Vehicle_Youthful_Driver_Training_Code'].str.replace(" ","")
    df_train['Policy_Method_Of_Payment']= df_train['Policy_Method_Of_Payment'].str.replace("-","")
    df_train['EEA_Agency_Type']= df_train['EEA_Agency_Type'].str.replace("-","")
    
    df_train = pd.get_dummies(df_train, columns=categorical_columns, prefix_sep="_")
    df_schm, df_train = df_schm.align(df_train, join='left', axis=1, fill_value=0)
    
    df_train.loc['median'] = df_train.median()
    X_mean = X_mean.append(df_train.loc['median'])
    
    Y = df_train[['EEA_Policy_Tenure','Annual_Premium','Loss_Amount','Loss_Ratio']]
    Y.loc['mean'] = Y.mean()
    Y_mean = Y_mean.append(Y.loc['mean'])


C:\Users\Vijay S Chauhan\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [93]:
X_mean.head()

,Annual_Premium,Claim_Count,Driver_Maximum_Age,Driver_Minimum_Age,Driver_Total,Driver_Total_Adult_Ages_50_64,Driver_Total_College_Ages_20_23,Driver_Total_Female,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Male,...,Vehicle_Youthful_Good_Student_Code_NotEligibleforGoodStudentCredit,pol_zip_code_1,pol_zip_code_2,pol_zip_code_3,pol_zip_code_4,pol_zip_code_5,pol_zip_code_6,pol_zip_code_7,pol_zip_code_8,pol_zip_code_9
median,307.40,0.0,60.0,59.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
median,312.70,0.0,58.0,57.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
median,333.90,0.0,49.0,48.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
median,339.20,0.0,49.0,49.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
median,325.42,0.0,51.0,50.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [94]:
#X_vijay
X_mean.drop(['EEA_Policy_Tenure','Annual_Premium','Loss_Amount','Frequency','Severity','Loss_Ratio'],axis=1, inplace=True)
X_mean.reset_index(inplace=True)
Y_mean.reset_index(inplace=True)

# Considering mean values for Policy tenure and annual premium
X_mean['EEA_Policy_Tenure'] = Y_mean['EEA_Policy_Tenure']
X_mean['Annual_Premium'] =Y_mean['Annual_Premium']

X_mean.drop(['index'],axis=1, inplace=True)
Y_mean.drop(['index'],axis=1, inplace=True)

Y_mean.drop(['EEA_Policy_Tenure','Annual_Premium','Loss_Amount'],axis=1, inplace=True)
#X_mean.drop(['Vehicle_Make_Year','Vehicle_Bodily_Injury_Limit','Driver_Total_Licensed_In_State','EEA_Prior_Bodily_Injury_Limit','Policy_Installment_Term','Vehicle_Youthful_Driver_Indicator','EEA_Liability_Coverage_Only_Indicator','Policy_Billing_Code','Vehicle_Youthful_Driver_Training_Code','Vehicle_Anti_Theft_Device'],axis=1, inplace=True)
X_mean.head()

,Claim_Count,Driver_Maximum_Age,Driver_Minimum_Age,Driver_Total,Driver_Total_Adult_Ages_50_64,Driver_Total_College_Ages_20_23,Driver_Total_Female,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Male,Driver_Total_Married,...,pol_zip_code_2,pol_zip_code_3,pol_zip_code_4,pol_zip_code_5,pol_zip_code_6,pol_zip_code_7,pol_zip_code_8,pol_zip_code_9,EEA_Policy_Tenure,Annual_Premium
0,0.0,60.0,59.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20.3042,306.95154
1,0.0,58.0,57.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,19.4427,312.70063
2,0.0,49.0,48.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8.7374,327.74981
3,0.0,49.0,49.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8.8064,331.32137
4,0.0,51.0,50.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8.7066,325.43480


In [95]:
Y_mean.head()

,Loss_Ratio
0,0.38136
1,1.19710
2,0.58121
3,0.44815
4,0.63082


In [96]:
X_mean.columns

Index(['Claim_Count', 'Driver_Maximum_Age', 'Driver_Minimum_Age',
       'Driver_Total', 'Driver_Total_Adult_Ages_50_64',
       'Driver_Total_College_Ages_20_23', 'Driver_Total_Female',
       'Driver_Total_Low_Middle_Adult_Ages_30_39', 'Driver_Total_Male',
       'Driver_Total_Married',
       ...
       'pol_zip_code_2', 'pol_zip_code_3', 'pol_zip_code_4', 'pol_zip_code_5',
       'pol_zip_code_6', 'pol_zip_code_7', 'pol_zip_code_8', 'pol_zip_code_9',
       'EEA_Policy_Tenure', 'Annual_Premium'],
      dtype='object', length=133)

In [97]:
# Devide dataset into training and testing dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_mean, Y_mean, test_size=0.33)

In [98]:
# scale input attributes for train dataset
scaler = preprocessing.StandardScaler()
scaler= scaler.fit(X_train)

# Scale input attributes for Train dataset
X_trainscaled = scaler.transform(X_train)

# Scale input attributes for Test dataset
X_testscaled = scaler.transform(X_test)

In [99]:
lgb_train = lgb.Dataset(X_trainscaled, label=Y_train)
params = {}
params['learning_rate'] = 0.1
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'rmse'
params['sub_feature'] = 0.9
params['num_leaves'] = 10
params['min_data'] = 100
params['max_depth'] = 10
lgb_tr = lgb.train(params, lgb_train, 1000)

In [100]:
# Ridge Regularization with scaling
y_pred=lgb_tr.predict(X_trainscaled)
rms = sqrt(mean_squared_error(y_pred, Y_train))
print('RMSE for training dataset for LGB model is :: ',rms)

RMSE for training dataset for LGB model is ::  0.3594437842971406


In [101]:
ytest_pred=lgb_tr.predict(X_testscaled)
t_rms = sqrt(mean_squared_error(ytest_pred, Y_test))
print('RMSE for testing dataset for LGB model is :: ',t_rms)

RMSE for testing dataset for LGB model is ::  0.34616842937454695


In [102]:
# Copy data in tabular format
dfObj_test = pd.DataFrame(columns=['FileName', 'PredictedLogRatio', 'ActualLogRatio','AbsoluteErrorLogRatio'])

In [103]:
path = r'.\Trainportfolio' # use your path
all_files = glob.glob(path + "/*.csv")
TX_mean = pd.DataFrame()
TY_mean = pd.DataFrame()

for filename in all_files:
    df_train = pd.read_csv(filename)
    
    df_train['Vehicle_Performance']= df_train['Vehicle_Performance'].str.replace(" ","")
    df_train['Policy_Billing_Code']= df_train['Policy_Billing_Code'].str.replace(" ","")
    df_train['Vehicle_Youthful_Good_Student_Code']= df_train['Vehicle_Youthful_Good_Student_Code'].str.replace(" ","")
    df_train['Vehicle_Anti_Theft_Device']= df_train['Vehicle_Anti_Theft_Device'].str.replace(" ","")
    df_train['Vehicle_Anti_Theft_Device']= df_train['Vehicle_Anti_Theft_Device'].str.replace("-","")
    df_train['Vehicle_Youthful_Driver_Training_Code']= df_train['Vehicle_Youthful_Driver_Training_Code'].str.replace(" ","")
    df_train['Policy_Method_Of_Payment']= df_train['Policy_Method_Of_Payment'].str.replace("-","")
    df_train['EEA_Agency_Type']= df_train['EEA_Agency_Type'].str.replace("-","")
    
    df_train = pd.get_dummies(df_train, columns=categorical_columns, prefix_sep="_")
    df_schm, df_train = df_schm.align(df_train, join='left', axis=1, fill_value=0)
    
    df_train.loc['median'] = df_train.median()
    TX_mean = TX_mean.append(df_train.loc['median'])
    
    Y = df_train[['EEA_Policy_Tenure','Annual_Premium','Loss_Amount','Loss_Ratio']]
    Y.loc['mean'] = Y.mean()
    TY_mean = TY_mean.append(Y.loc['mean'])
    
    dfObj_test = dfObj_test.append({'FileName': filename[filename.rfind('\\')+1:], 'PredictedLogRatio': 0, 'ActualLogRatio': 0, 
                          'AbsoluteErrorLogRatio': 0}, ignore_index=True)


C:\Users\Vijay S Chauhan\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [104]:
#X_vijay
TX_mean.drop(['EEA_Policy_Tenure','Annual_Premium','Loss_Amount','Frequency','Severity','Loss_Ratio'],axis=1, inplace=True)
TX_mean.reset_index(inplace=True)
TY_mean.reset_index(inplace=True)

# Considering mean values for Policy tenure and annual premium
TX_mean['EEA_Policy_Tenure'] = TY_mean['EEA_Policy_Tenure']
TX_mean['Annual_Premium'] =TY_mean['Annual_Premium']

TX_mean.drop(['index'],axis=1, inplace=True)
TY_mean.drop(['index'],axis=1, inplace=True)

TY_mean.drop(['EEA_Policy_Tenure','Annual_Premium','Loss_Amount'],axis=1, inplace=True)
TX_mean.head()

,Claim_Count,Driver_Maximum_Age,Driver_Minimum_Age,Driver_Total,Driver_Total_Adult_Ages_50_64,Driver_Total_College_Ages_20_23,Driver_Total_Female,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Male,Driver_Total_Married,...,pol_zip_code_2,pol_zip_code_3,pol_zip_code_4,pol_zip_code_5,pol_zip_code_6,pol_zip_code_7,pol_zip_code_8,pol_zip_code_9,EEA_Policy_Tenure,Annual_Premium
0,0.0,47.0,46.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.750070,333.810344
1,0.0,47.0,46.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.003446,331.295373
2,0.0,47.0,45.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.845212,333.993435
3,0.0,47.0,46.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.928285,330.639129
4,0.0,47.0,46.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.009696,335.539706


In [105]:
# Scale input attributes for Test dataset
TX_testscaled = scaler.transform(TX_mean)

In [106]:
dfObj_test['ActualLogRatio'] = np.log(TY_mean)
dfObj_test['PredictedLogRatio'] = np.log(lgb_tr.predict(TX_testscaled))
dfObj_test['AbsoluteErrorLogRatio'] = dfObj_test['ActualLogRatio'] - dfObj_test['PredictedLogRatio']
yftest_pred=lgb_tr.predict(TX_testscaled)
tf_rms = sqrt(mean_squared_error(yftest_pred, TY_mean))
print('RMSE for testing dataset for LGB model is :: ',tf_rms)

RMSE for testing dataset for LGB model is ::  0.1704675753410191


In [107]:
dfObj_test.head()

,FileName,PredictedLogRatio,ActualLogRatio,AbsoluteErrorLogRatio
0,Test1.csv,-0.479266,0.028678,0.507944
1,Test10.csv,-0.428747,-0.244286,0.184461
2,Test11.csv,-0.453688,-0.349316,0.104372
3,Test12.csv,-0.428747,-0.163686,0.265061
4,Test13.csv,-0.427359,-0.229217,0.198143


In [108]:
dfObj_test.to_csv('PortwiseLGBResults.csv',index = False)

In [109]:
dfObj_test.head()

,FileName,PredictedLogRatio,ActualLogRatio,AbsoluteErrorLogRatio
0,Test1.csv,-0.479266,0.028678,0.507944
1,Test10.csv,-0.428747,-0.244286,0.184461
2,Test11.csv,-0.453688,-0.349316,0.104372
3,Test12.csv,-0.428747,-0.163686,0.265061
4,Test13.csv,-0.427359,-0.229217,0.198143


In [110]:
# Predicting values for actual files

df_schm1 = df_schm.drop(['Loss_Amount','Frequency','Severity','Loss_Ratio'],axis=1)
# Copy data in tabular format
fObj_test = pd.DataFrame(columns=['ID', 'ln_LR'])

path = r'.\Spring_testing_portfolios_Upd' # use your path   N-folder

#path = r'.\N-folder'

all_files = glob.glob(path + "/*.csv")
FTX_mean = pd.DataFrame()
FTY_mean = pd.DataFrame()

for filename in all_files:
    df_pred = pd.read_csv(filename)

    df_pred['Vehicle_Performance']= df_pred['Vehicle_Performance'].str.replace(" ","")
    df_pred['Policy_Billing_Code']= df_pred['Policy_Billing_Code'].str.replace(" ","")
    df_pred['Vehicle_Youthful_Good_Student_Code']= df_pred['Vehicle_Youthful_Good_Student_Code'].str.replace(" ","")
    df_pred['Vehicle_Anti_Theft_Device']= df_pred['Vehicle_Anti_Theft_Device'].str.replace(" ","")
    df_pred['Vehicle_Anti_Theft_Device']= df_pred['Vehicle_Anti_Theft_Device'].str.replace("-","")
    df_pred['Vehicle_Youthful_Driver_Training_Code']= df_pred['Vehicle_Youthful_Driver_Training_Code'].str.replace(" ","")
    df_pred['Policy_Method_Of_Payment']= df_pred['Policy_Method_Of_Payment'].str.replace("-","")
    df_pred['EEA_Agency_Type']= df_pred['EEA_Agency_Type'].str.replace("-","")
    
    df_pred = pd.get_dummies(df_pred, columns=categorical_columns, prefix_sep="_")
    df_schm1, df_pred = df_schm1.align(df_pred, join='left', axis=1, fill_value=0)
    
    df_pred.loc['median'] = df_pred.median()
    FTX_mean = FTX_mean.append(df_pred.loc['median'])
    
    Y = df_pred[['EEA_Policy_Tenure','Annual_Premium']]
    Y.loc['mean'] = Y.mean()
    FTY_mean = FTY_mean.append(Y.loc['mean']) 
    
    fname = filename[filename.rfind('\\')+1:]
    
    fObj_test = fObj_test.append({'ID': fname, 'ln_LR': 0}, ignore_index=True)

    

FTX_mean.drop(['EEA_Policy_Tenure','Annual_Premium'],axis=1, inplace=True)
FTX_mean.reset_index(inplace=True)
FTY_mean.reset_index(inplace=True)

# Considering mean values for Policy tenure and annual premium
FTX_mean['EEA_Policy_Tenure'] = FTY_mean['EEA_Policy_Tenure']
FTX_mean['Annual_Premium'] =FTY_mean['Annual_Premium']

FTX_mean.drop(['index'],axis=1, inplace=True)
FTY_mean.drop(['index'],axis=1, inplace=True)

FTY_mean.drop(['EEA_Policy_Tenure','Annual_Premium'],axis=1, inplace=True)

# Scale input attributes for Test dataset
FTX_testscaled = scaler.transform(FTX_mean)

fObj_test['ln_LR'] = np.log(lgb_tr.predict(FTX_testscaled))
print(np.log(lgb_tr.predict(FTX_testscaled)))

KeyError: "['pol_zip_code'] not in index"

In [85]:
fObj_test.head()
#result = df.sort(['A', 'B'], ascending=[1, 0])
port = fObj_test.sort_values(by=['ID'])
port.to_csv('FinalLGBPrediction.csv',index = False)